In [1]:
import sys
sys.path.append(r"C:\Users\SHAHID\shahid\SHAHID")

In [2]:
import os
from stlf_torch_kit import  DataLoadeing
import torch
import pandas as pd
import numpy as np
import pickle, time
from stlf_torch_kit import univariate_multi_step
from stlf_torch_kit import SaveBestModel, PlotLossCurves, LoadModel, train, TestModel, BatchGenerator, results
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary


In [ ]:
import pandas as pd

# Load your Excel file
df = pd.read_csv(r'C:\Users\SHAHID\shahid (ML-lab)\cleaned_citylearn_data_all_outliers_handled.csv')

import pandas as pd

# Load the CSV file
#df = pd.read_csv("your_file.csv")  # Replace with your actual file name

# Keep only the 'non_shiftable_load' column
df_filtered = df[["non_shiftable_load"]]

# Save the filtered dataframe to a new CSV
df_filtered.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\Raw data.csv', index=False)



In [71]:
import pandas as pd

# Step 1: Load the CSV file
df = pd.read_csv(r'C:\Users\SHAHID\shahid (ML-lab)\Raw data.csv')  # Replace with your actual file name

# Step 2: Keep only the non_shiftable_load column
df = df[["non_shiftable_load"]]

# Step 3: Generate hourly datetime range
start_date = "2022-08-01 01:00"
datetime_range = pd.date_range(start=start_date, periods=8760, freq='H')

# Step 4: Add datetime column
df["datetime"] = datetime_range

# Step 5: Reorder columns (datetime first)
df = df[["datetime", "non_shiftable_load"]]

# Step 6: Save the new DataFrame
df.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\non_shiftable_with_datetime.csv', index=False)

print("✅ File 'non_shiftable_with_datetime.csv' created successfully.")


✅ File 'non_shiftable_with_datetime.csv' created successfully.


In [73]:
import pandas as pd
import numpy as np
from typing import List
from pandas.tseries import offsets
from pandas.tseries.frequencies import to_offset

# ---- Time Feature Base Class ---- #
class TimeFeature:
    def __call__(self, index: pd.DatetimeIndex) -> np.ndarray:
        raise NotImplementedError

    def __repr__(self):
        return self.__class__.__name__ + "()"

# ---- Concrete Time Features ---- #
class SecondOfMinute(TimeFeature):
    def __call__(self, index): return index.dt.second / 59.0 - 0.5

class MinuteOfHour(TimeFeature):
    def __call__(self, index): return index.dt.minute / 59.0 - 0.5

class HourOfDay(TimeFeature):
    def __call__(self, index): return index.dt.hour / 23.0 - 0.5

class DayOfWeek(TimeFeature):
    def __call__(self, index): return index.dt.dayofweek / 6.0 - 0.5

class DayOfMonth(TimeFeature):
    def __call__(self, index): return (index.dt.day - 1) / 30.0 - 0.5

class DayOfYear(TimeFeature):
    def __call__(self, index): return (index.dt.dayofyear - 1) / 365.0 - 0.5

class MonthOfYear(TimeFeature):
    def __call__(self, index): return (index.dt.month - 1) / 11.0 - 0.5

class WeekOfYear(TimeFeature):
    def __call__(self, index): return (index.dt.isocalendar().week - 1) / 52.0 - 0.5

# ---- Feature Extractor from Frequency String ---- #
def time_features_from_frequency_str(freq_str: str) -> List[TimeFeature]:
    features_by_offsets = {
        offsets.YearEnd: [],
        offsets.QuarterEnd: [MonthOfYear],
        offsets.MonthEnd: [MonthOfYear],
        offsets.Week: [DayOfMonth, WeekOfYear],
        offsets.Day: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.BusinessDay: [DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Hour: [HourOfDay, DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Minute: [MinuteOfHour, HourOfDay, DayOfWeek, DayOfMonth, DayOfYear],
        offsets.Second: [SecondOfMinute, MinuteOfHour, HourOfDay, DayOfWeek, DayOfMonth, DayOfYear],
    }

    offset = to_offset(freq_str)

    for offset_type, feature_classes in features_by_offsets.items():
        if isinstance(offset, offset_type):
            return [cls() for cls in feature_classes]

    raise RuntimeError(f"Unsupported frequency: {freq_str}")

# ---- Time Feature Wrapper ---- #
def time_features(dates: pd.DataFrame, timeenc=1, freq='h'):
    if timeenc == 0:
        dates['month'] = dates['Datetime'].apply(lambda x: x.month)
        dates['day'] = dates['Datetime'].apply(lambda x: x.day)
        dates['weekday'] = dates['Datetime'].apply(lambda x: x.weekday())
        dates['hour'] = dates['Datetime'].apply(lambda x: x.hour)
        dates['minute'] = dates['Datetime'].apply(lambda x: x.minute)
        dates['minute'] = dates['Datetime'].map(lambda x: x // 15)
        freq_map = {
            'y': [],
            'm': ['month'],
            'w': ['month'],
            'd': ['month', 'day', 'weekday'],
            'b': ['month', 'day', 'weekday'],
            'h': ['month', 'day', 'weekday', 'hour'],
            't': ['month', 'day', 'weekday', 'hour', 'minute'],
        }
        return dates[freq_map[freq.lower()]].values
    elif timeenc == 1:
        datetime_index = pd.to_datetime(dates['Datetime'])
        return np.vstack([feat(datetime_index) for feat in time_features_from_frequency_str(freq)]).transpose(1, 0)

# ---- Main: Load CSV and Apply Time Features ---- #
# Step 1: Load your CSV (with non_shiftable_load column)
df = pd.read_csv(r'C:\Users\SHAHID\shahid (ML-lab)\non_shiftable_with_datetime.csv')  # Replace with your actual CSV name

# Step 2: Create datetime range and add to DataFrame
start_date = "2022-08-01 00:00"
datetime_range = pd.date_range(start=start_date, periods=len(df), freq='H')
df['Datetime'] = datetime_range

# Step 3: Extract time features
time_feats = time_features(df[['Datetime']], timeenc=1, freq='h')

# Step 4: Combine time features into DataFrame
time_feat_names = ['hour_of_day', 'day_of_week', 'day_of_month', 'day_of_year']
time_feat_df = pd.DataFrame(time_feats, columns=time_feat_names)

# Step 5: Final dataset with non_shiftable_load + time features
final_df = pd.concat([df[["Datetime", "non_shiftable_load"]], time_feat_df], axis=1)

# Step 6: Save the final DataFrame
final_df.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\non_shiftable_with_time_features.csv', index=False)

print("✅ Final dataset saved as 'non_shiftable_with_time_features.csv'")


✅ Final dataset saved as 'non_shiftable_with_time_features.csv'


In [80]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Step 1: Load dataset
df = pd.read_csv(r'C:\Users\SHAHID\shahid (ML-lab)\non_shiftable_with_time_features.csv')

# Step 2: Drop 'Datetime' column only
df = df.drop(columns=["Datetime"])

# Step 3: Split into 90/5/5
total_len = len(df)
train_end = int(0.9 * total_len)
val_end = int(0.95 * total_len)

train_df = df.iloc[:train_end].copy()
val_df = df.iloc[train_end:val_end].copy()
test_df = df.iloc[val_end:].copy()

# Step 4: Scale only 'non_shiftable_load'
scaler = MinMaxScaler()
train_df['non_shiftable_load'] = scaler.fit_transform(train_df[['non_shiftable_load']])
val_df['non_shiftable_load'] = scaler.transform(val_df[['non_shiftable_load']])
test_df['non_shiftable_load'] = scaler.transform(test_df[['non_shiftable_load']])

# Step 5: Save results
train_df.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer\train.csv', index=False)
val_df.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer\validation.csv', index=False)
test_df.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer\test.csv', index=False)

print("✅ Load scaled. Time features untouched. Files saved.")


✅ Load scaled. Time features untouched. Files saved.


In [97]:
import pandas as pd

# Step 1: Load your dataset
df = pd.read_csv(r'C:\Users\SHAHID\shahid (ML-lab)\cleaned_citylearn_data_all_outliers_handled.csv')

# Step 2: Define fixed columns to drop
fixed_cols_to_drop = ['non_shiftable_load', 'day_type', 'hour', 'month']

# Step 3: Dynamically find all columns containing 'predicted'
predicted_cols_to_drop = [col for col in df.columns if 'predicted' in col]

# Step 4: Combine all columns to drop
all_cols_to_drop = fixed_cols_to_drop + predicted_cols_to_drop

# Step 5: Drop those columns
df_cleaned = df.drop(columns=all_cols_to_drop)

# Step 6: Save cleaned dataset
df_cleaned.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer\features_without_predicted.csv', index=False)

print("✅ Columns removed and cleaned file saved successfully.")


✅ Columns removed and cleaned file saved successfully.


In [100]:
import pandas as pd

# Load both datasets
features_cleaned = pd.read_csv(r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer\features_without_predicted.csv')
full_with_time = pd.read_csv(r'C:\Users\SHAHID\shahid (ML-lab)\non_shiftable_with_time_features.csv')

# Concatenate both DataFrames (side by side)
df_final = pd.concat([ full_with_time.reset_index(drop=True),features_cleaned.reset_index(drop=True)], axis=1)

# Save the result
df_final.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer\final_combined_features.csv', index=False)

print("✅ Files concatenated successfully and saved as final_combined_features.csv")


✅ Files concatenated successfully and saved as final_combined_features.csv


In [101]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Step 1: Load dataset
df = pd.read_csv(r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer\final_combined_features.csv')  # Replace with correct file path

# Step 2: Remove 'Datetime' column if exists
if 'Datetime' in df.columns:
    df = df.drop(columns=['Datetime'])

# Step 3: Define non-scaled time feature columns
non_scaled_cols = ['hour_of_day', 'day_of_week', 'day_of_month', 'day_of_year']

# Step 4: Separate scaled and unscaled parts
scaled_cols = [col for col in df.columns if col not in non_scaled_cols]
df_scaled_part = df[scaled_cols]
df_unscaled_part = df[non_scaled_cols]

# Step 5: Split into 90%, 5%, 5%
total_len = len(df)
train_end = int(0.9 * total_len)
val_end = int(0.95 * total_len)

# Split both parts
train_scaled = df_scaled_part.iloc[:train_end]
val_scaled = df_scaled_part.iloc[train_end:val_end]
test_scaled = df_scaled_part.iloc[val_end:]

train_unscaled = df_unscaled_part.iloc[:train_end]
val_unscaled = df_unscaled_part.iloc[train_end:val_end]
test_unscaled = df_unscaled_part.iloc[val_end:]

# Step 6: Apply MinMaxScaler only to scaled columns
scaler = MinMaxScaler()
train_scaled_trans = scaler.fit_transform(train_scaled)
val_scaled_trans = scaler.transform(val_scaled)
test_scaled_trans = scaler.transform(test_scaled)

# Step 7: Convert back to DataFrames
train_final = pd.concat([
    pd.DataFrame(train_scaled_trans, columns=scaled_cols).reset_index(drop=True),
    train_unscaled.reset_index(drop=True)
], axis=1)

val_final = pd.concat([
    pd.DataFrame(val_scaled_trans, columns=scaled_cols).reset_index(drop=True),
    val_unscaled.reset_index(drop=True)
], axis=1)

test_final = pd.concat([
    pd.DataFrame(test_scaled_trans, columns=scaled_cols).reset_index(drop=True),
    test_unscaled.reset_index(drop=True)
], axis=1)

# Step 8: Save to CSV
train_final.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer\train1.csv', index=False)
val_final.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer\validation1.csv', index=False)
test_final.to_csv(r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer\test1.csv', index=False)

print("✅ Done! Scaled selected columns, kept time features unchanged, and saved all splits.")


✅ Done! Scaled selected columns, kept time features unchanged, and saved all splits.


In [3]:
from platform import python_version
print(python_version())

3.7.0


# Data Loading

In [102]:
path_dataset =r'C:\Users\SHAHID\shahid (ML-lab)\citylearn datset for informer' #Edit
path_tr = os.path.join(path_dataset, 'train1.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.values
path_v = os.path.join(path_dataset, 'validation1.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.values
path_te = os.path.join(path_dataset, 'test1.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.values 

path_scaler = os.path.join(path_dataset, 'scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((7884, 12), (438, 12), (438, 12))

In [103]:
time_steps=24 #look back or sequence length, lag, window size #Edit
target_len = 1 #how much steps do you wana forecast #Edit
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=target_len)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=target_len)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=target_len)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.14592647552490234 sec


In [104]:
train_X.shape,train_y.shape,validation_X.shape, validation_y.shape,test_X.shape, test_y.shape

((7859, 24, 12), (7859, 1), (413, 24, 12), (413, 1), (413, 24, 12), (413, 1))

# Proposed Model

#### LSTM

In [56]:
# major edit
class LSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.lstm = nn.LSTM(14, 20, 1, batch_first=True).to(self.device) # num-features, node/units, num-layers
        self.fc = nn.Linear(20, 1).to(self.device) #(in,out)

    def forward(self, x):
        out, _ = self.lstm(x) # _=(h,c)
        out = self.fc(out[:, -1, :]) 
        return out

In [57]:
model3 = LSTMModel()
print(summary(model, input_size=(64, 24, 14)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN1DModel                               [64, 1]                   --
├─Conv1d: 1-1                            [64, 32, 22]              1,376
├─Linear: 1-2                            [64, 1]                   705
Total params: 2,081
Trainable params: 2,081
Non-trainable params: 0
Total mult-adds (M): 1.98
Input size (MB): 0.09
Forward/backward pass size (MB): 0.36
Params size (MB): 0.01
Estimated Total Size (MB): 0.46


#### CNN
<table style="border: 1px solid black; border-collapse: collapse; width: 100%;">
    <thead>
        <tr style="background-color: #f2f2f2;">
            <th style="border: 1px solid black; padding: 8px;">Layer</th>
            <th style="border: 1px solid black; padding: 8px;">Expected Input Shape</th>
            <th style="border: 1px solid black; padding: 8px;">Common Adjustments Needed</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="border: 1px solid black; padding: 8px;">Conv1d</td>
            <td style="border: 1px solid black; padding: 8px;">(batch_size, channels, sequence_length)</td>
            <td style="border: 1px solid black; padding: 8px;">Permute input from (batch, seq, feat) to (batch, feat, seq)</td>
        </tr>
        <tr>
            <td style="border: 1px solid black; padding: 8px;">LSTM</td>
            <td style="border: 1px solid black; padding: 8px;">(batch_size, sequence_length, features)</td>
            <td style="border: 1px solid black; padding: 8px;">No permutation if batch_first=True</td>
        </tr>
    </tbody>
</table>



In [97]:
class CNN1DModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.conv1 = nn.Conv1d(in_channels=14, out_channels=32, kernel_size=3).to(self.device)
        self.fc = nn.Linear(32*22, 1).to(self.device) #32 channels ao 22 features dy chy kernel size 3 na bad kam shwal

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = torch.relu(self.conv1(x))  
        x = x.reshape(x.shape[0], -1)
        out = self.fc(x) 
        return out


In [98]:
model1 = CNN1DModel()
print(summary(model1, input_size=(64, 24, 14)))
# summary(model, input_size=(64, 24, 1), col_names=["input_size", "output_size", "num_params", "kernel_size"])

Layer (type:depth-idx)                   Output Shape              Param #
CNN1DModel                               [64, 1]                   --
├─Conv1d: 1-1                            [64, 32, 22]              1,376
├─Linear: 1-2                            [64, 1]                   705
Total params: 2,081
Trainable params: 2,081
Non-trainable params: 0
Total mult-adds (M): 1.98
Input size (MB): 0.09
Forward/backward pass size (MB): 0.36
Params size (MB): 0.01
Estimated Total Size (MB): 0.46


In [99]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from math import sqrt

class ProbSparseAttention(nn.Module):
    def __init__(self, mask_flag=False, factor=5, scale=None, attention_dropout=0.1):
        super(ProbSparseAttention, self).__init__()
        self.factor = factor
        self.scale = scale
        self.mask_flag = mask_flag
        self.dropout = nn.Dropout(attention_dropout)

    def _prob_QK(self, Q, K, sample_k, n_top):
        B, H, L_K, E = K.shape
        _, _, L_Q, _ = Q.shape

        K_expand = K.unsqueeze(-3).expand(B, H, L_Q, L_K, E)
        index_sample = torch.randint(L_K, (L_Q, sample_k))
        K_sample = K_expand[:, :, torch.arange(L_Q).unsqueeze(1), index_sample, :]
        Q_K_sample = torch.matmul(Q.unsqueeze(-2), K_sample.transpose(-2, -1)).squeeze(-2)

        M = Q_K_sample.max(-1)[0] - torch.div(Q_K_sample.sum(-1), L_K)
        M_top = M.topk(n_top, sorted=False)[1]

        Q_reduce = Q[torch.arange(B)[:, None, None],
                    torch.arange(H)[None, :, None],
                    M_top, :]
        Q_K = torch.matmul(Q_reduce, K.transpose(-2, -1))
        return Q_K, M_top

    def forward(self, queries, keys, values, attn_mask=None):
        B, L_Q, H, D = queries.shape
        _, L_K, _, _ = keys.shape

        queries = queries.transpose(1, 2)  # [B, H, L_Q, D]
        keys = keys.transpose(1, 2)       # [B, H, L_K, D]
        values = values.transpose(1, 2)   # [B, H, L_K, D]

        U_part = min(self.factor * np.ceil(np.log(L_K)).astype(int), L_K)
        u = min(self.factor * np.ceil(np.log(L_Q)).astype(int), L_Q)
        
        scores_top, index = self._prob_QK(queries, keys, sample_k=U_part, n_top=u)

        scale = self.scale or 1./sqrt(D)
        scores_top = scores_top * scale

        attn = torch.softmax(scores_top, dim=-1)
        attn = self.dropout(attn)
        
        context = torch.matmul(attn, values)  # [B, H, u, D]
        
        output = torch.zeros_like(queries).to(queries.device)
        output[torch.arange(B)[:, None, None],
               torch.arange(H)[None, :, None],
               index, :] = context
        return output.transpose(1, 2).contiguous(), attn

class InformerSTLF(nn.Module):
    def __init__(self, enc_in=21, d_model=128, d_ff=256, n_heads=8, e_layers=3, dropout=0.2):
        super(InformerSTLF, self).__init__()
        self.n_heads = n_heads  # Store as class attribute
        
        # Input Embedding
        self.enc_embedding = nn.Sequential(
            nn.Linear(enc_in, d_model//2),
            nn.GELU(),
            nn.Linear(d_model//2, d_model)
        )
        self.pos_embedding = nn.Parameter(torch.randn(1, 24, d_model))
        self.dropout = nn.Dropout(dropout)
        
        # ConvLayer without downsampling
        self.conv = nn.Sequential(
            nn.Conv1d(d_model, d_model, kernel_size=3, padding=1),
            nn.BatchNorm1d(d_model),
            nn.GELU()
        )
        
        # Encoder Layers
        self.encoder_layers = nn.ModuleList([
            nn.ModuleList([
                ProbSparseAttention(mask_flag=False, factor=5, scale=None, attention_dropout=dropout),
                nn.LayerNorm(d_model),
                nn.Sequential(
                    nn.Linear(d_model, d_ff),
                    nn.GELU(),
                    nn.Dropout(dropout),
                    nn.Linear(d_ff, d_model)
                ),
                nn.LayerNorm(d_model)
            ]) for _ in range(e_layers)
        ])
        
        self.projection = nn.Sequential(
            nn.Linear(d_model, d_model//2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(d_model//2, 1)
        )

    def forward(self, x):
        # Input shape: [batch_size, 24, 21]
        x = self.enc_embedding(x) + self.pos_embedding
        x = self.dropout(x)
        
        # ConvLayer (no length reduction)
        x_conv = x.permute(0, 2, 1)  # [B, D, L]
        x_conv = self.conv(x_conv)
        x = x + x_conv.permute(0, 2, 1)  # Residual connection
        
        # Encoder
        for attn, norm1, ff, norm2 in self.encoder_layers:
            # Attention Block
            residual = x
            x = x.unsqueeze(1).expand(-1, self.n_heads, -1, -1)  # Now using stored n_heads
            x, _ = attn(x, x, x)
            x = x.mean(dim=1)  # Average over heads
            x = norm1(x + residual)
            
            # Feed Forward Block
            residual = x
            x = ff(x)
            x = norm2(x + residual)
        
        # Final Prediction
        x = x[:, -1, :]  # Last timestep
        return self.projection(x)

In [100]:
model7 = InformerSTLF(
    enc_in=14,          # Input features
    d_model=128,        # Model dimension
    d_ff=256,           # Feed-forward dimension
    n_heads=8,          # Attention heads
    e_layers=3,         # Encoder layers
    dropout=0.2         # Dropout rate
)

# Check summary
print(summary(model7, input_size=(64, 24, 14)))

Layer (type:depth-idx)                   Output Shape              Param #
InformerSTLF                             [64, 1]                   3,072
├─Sequential: 1-1                        [64, 24, 128]             --
│    └─Linear: 2-1                       [64, 24, 64]              960
│    └─GELU: 2-2                         [64, 24, 64]              --
│    └─Linear: 2-3                       [64, 24, 128]             8,320
├─Dropout: 1-2                           [64, 24, 128]             --
├─Sequential: 1-3                        [64, 128, 24]             --
│    └─Conv1d: 2-4                       [64, 128, 24]             49,280
│    └─BatchNorm1d: 2-5                  [64, 128, 24]             256
│    └─GELU: 2-6                         [64, 128, 24]             --
├─ModuleList: 1-4                        --                        --
│    └─ModuleList: 2-7                   --                        --
│    │    └─ProbSparseAttention: 3-1     [64, 8, 24, 128]          --
│  

#### Mostly Cited

In [85]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

# Define the model
class MLP(nn.Module):
    def __init__(self, input_size=24*14):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 1)
        
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, x):
        x = x.to(self.device)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [86]:
# Create and summarize the model
model6 = MLP(input_size=24*14)
print(summary(model, input_size=(32, 24, 14)))

Layer (type:depth-idx)                   Output Shape              Param #
CNNLSTMModel                             [32, 1]                   --
├─Conv1d: 1-1                            [32, 64, 22]              2,752
├─LSTM: 1-2                              [32, 22, 50]              64,000
├─Linear: 1-3                            [32, 20]                  1,020
├─Linear: 1-4                            [32, 1]                   21
Total params: 67,793
Trainable params: 67,793
Non-trainable params: 0
Total mult-adds (M): 47.03
Input size (MB): 0.04
Forward/backward pass size (MB): 0.65
Params size (MB): 0.27
Estimated Total Size (MB): 0.96


In [109]:
class MOSTLY_CITED1(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.lstm = nn.LSTM(14, 20, 2, batch_first=True).to(self.device)
        self.fc = nn.Linear(20, 1).to(self.device) 

    def forward(self, x):
        out, _ = self.lstm(x) # _=(h,c)
        out = torch.sigmoid(self.fc(out[:, -1, :]) )
        return out


In [105]:
class MOSTLY_CITED(nn.Module):
    def __init__(self, input_size=12, hidden_size=20, num_layers=2):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        
        # Move all parameters to device
        self.to(self.device)

    def forward(self, x):
        # Move input to same device as model
        x = x.to(self.device)
        out, _ = self.lstm(x)  # out shape: (batch, seq_len, hidden_size)
        out = self.fc(out[:, -1, :])  # Take last timestep
        return out

In [106]:
model = MOSTLY_CITED()
print(summary(model, input_size=(32, 24, 12)))

Layer (type:depth-idx)                   Output Shape              Param #
MOSTLY_CITED                             [32, 1]                   --
├─LSTM: 1-1                              [32, 24, 20]              6,080
├─Linear: 1-2                            [32, 1]                   21
Total params: 6,101
Trainable params: 6,101
Non-trainable params: 0
Total mult-adds (M): 4.67
Input size (MB): 0.04
Forward/backward pass size (MB): 0.12
Params size (MB): 0.02
Estimated Total Size (MB): 0.18


In [76]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, input_channels=14, time_steps=24):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=16, kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=16, kernel_size=2)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(16*(time_steps-2), 1)  # -2 because of two conv layers with kernel_size=2
        
        # Move model to appropriate device
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, x):
        x = x.to(self.device)
        # PyTorch expects (batch, channels, seq_len) for Conv1d
        x = x.permute(0, 2, 1)  # Change from (batch, seq_len, channels) to (batch, channels, seq_len)
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = self.flatten(x)
        x = self.fc(x)
        return x

In [77]:
model5 = CNN(input_channels=14, time_steps=24)
print(summary(model5, input_size=(32, 24, 14)))

Layer (type:depth-idx)                   Output Shape              Param #
CNNLSTMModel                             [32, 1]                   --
├─Conv1d: 1-1                            [32, 64, 22]              2,752
├─LSTM: 1-2                              [32, 22, 50]              64,000
├─Linear: 1-3                            [32, 20]                  1,020
├─Linear: 1-4                            [32, 1]                   21
Total params: 67,793
Trainable params: 67,793
Non-trainable params: 0
Total mult-adds (M): 47.03
Input size (MB): 0.04
Forward/backward pass size (MB): 0.65
Params size (MB): 0.27
Estimated Total Size (MB): 0.96


In [44]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size=16, hidden_size1=8, hidden_size2=20):
        super().__init__()
        self.rnn1 = nn.RNN(input_size=input_size, 
                          hidden_size=hidden_size1,
                          batch_first=True,
                          nonlinearity='relu')
        self.rnn2 = nn.RNN(input_size=hidden_size1,
                          hidden_size=hidden_size2,
                          batch_first=True,
                          nonlinearity='relu')
        self.fc = nn.Linear(hidden_size2, 1)
        
        # Move model to appropriate device
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, x):
        x = x.to(self.device)
        # First RNN layer (returns all sequences)
        x, _ = self.rnn1(x)
        # Second RNN layer (returns only last output)
        x, _ = self.rnn2(x)
        # Take the last time step's output
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [45]:
model2 = RNN(input_size=16)
print(summary(model2, input_size=(32, 24, 16)))

Layer (type:depth-idx)                   Output Shape              Param #
RNN                                      [32, 1]                   --
├─RNN: 1-1                               [32, 24, 8]               208
├─RNN: 1-2                               [32, 24, 20]              600
├─Linear: 1-3                            [32, 1]                   21
Total params: 829
Trainable params: 829
Non-trainable params: 0
Total mult-adds (M): 0.62
Input size (MB): 0.05
Forward/backward pass size (MB): 0.17
Params size (MB): 0.00
Estimated Total Size (MB): 0.22


In [65]:
class CNNLSTMModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.device = ('cuda' if torch.cuda.is_available() else 'cpu')
        self.conv1 = nn.Conv1d(in_channels=14, out_channels=64, kernel_size=3).to(self.device)
       
        self.lstm = nn.LSTM(64, 50, 3, batch_first=True).to(self.device)
        self.fc = nn.Linear(50, 20).to(self.device) 
        self.fc1 = nn.Linear(20, 1).to(self.device) 


    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = torch.relu(self.conv1(x))
        
        x = x.permute(0, 2, 1)
        out, _ = self.lstm(x) # _=(h,c)
        out = self.fc(out[:, -1, :])
        out = self.fc1(out)
        return out


In [67]:
model4 = CNNLSTMModel()
print(summary(model4, input_size=(64, 24, 14)))

Layer (type:depth-idx)                   Output Shape              Param #
CNNLSTMModel                             [64, 1]                   --
├─Conv1d: 1-1                            [64, 64, 22]              2,752
├─LSTM: 1-2                              [64, 22, 50]              64,000
├─Linear: 1-3                            [64, 20]                  1,020
├─Linear: 1-4                            [64, 1]                   21
Total params: 67,793
Trainable params: 67,793
Non-trainable params: 0
Total mult-adds (M): 94.05
Input size (MB): 0.09
Forward/backward pass size (MB): 1.29
Params size (MB): 0.27
Estimated Total Size (MB): 1.65


In [28]:
import torch
import torch.nn as nn

class LSTMForecastModel(nn.Module):
    def __init__(self, time_steps, num_features):
        super(LSTMForecastModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size=num_features, hidden_size=8, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=8, hidden_size=20, batch_first=True)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(20, 1)  # final LSTM layer output size is 20

    def forward(self, x):
        # x: [batch_size, time_steps, num_features]
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = x[:, -1, :]  # Take the last time step output
        x = self.flatten(x)
        out = self.fc(x)
        return out


In [29]:
time_steps = 24
num_features = 29
# Instantiate the model
model8 = LSTMForecastModel(time_steps, num_features)
print(summary(model8, input_size=(64, 24, 29)))

Layer (type:depth-idx)                   Output Shape              Param #
LSTMForecastModel                        [64, 1]                   --
├─LSTM: 1-1                              [64, 24, 8]               1,248
├─LSTM: 1-2                              [64, 24, 20]              2,400
├─Flatten: 1-3                           [64, 20]                  --
├─Linear: 1-4                            [64, 1]                   21
Total params: 3,669
Trainable params: 3,669
Non-trainable params: 0
Total mult-adds (M): 5.60
Input size (MB): 0.18
Forward/backward pass size (MB): 0.34
Params size (MB): 0.01
Estimated Total Size (MB): 0.54


In [11]:
'cuda' if torch.cuda.is_available() else 'cpu'

'cpu'

# instances

In [30]:
model8

LSTMForecastModel(
  (lstm1): LSTM(29, 8, batch_first=True)
  (lstm2): LSTM(8, 20, batch_first=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=20, out_features=1, bias=True)
)

In [107]:
model = MOSTLY_CITED()
#model=LSTMForecastModel(time_steps, num_features)
#model = RNN(input_size=16)
criterion = nn.MSELoss() #Edit, don't change

save_best_model = SaveBestModel()
Plot_Loss=PlotLossCurves()
load_model=LoadModel()

# Learning Rate & Optimizer

In [108]:
lr=0.001 # Edit
#optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=lr) #Edit

# Check Device

In [109]:
def get_model_device(model):
    return next(model.parameters()).device
device = get_model_device(model)
print("Model is on device:", device)

Model is on device: cpu


# Training

#### Path & other Stuffs

In [110]:
start_epoch = 1
num_epochs = 50#Edit
best_model_path=r'C:\Users\SHAHID\shahid (ML-lab)\pro_result'+str('\\') #Edit
fig_path=r'C:\Users\SHAHID\shahid (ML-lab)\pro_result' #Edit
train_data_loader, validation_data_loader, test_data_loader = DataLoadeing(train_X ,
                                                                           train_y, 
                                                                           validation_X, 
                                                                           validation_y, 
                                                                           test_X, 
                                                                           test_y, 
                                                                           batch_size=32) #Batch_Size Edit

#### Instances

In [111]:
criterion = nn.MSELoss()  # Edit, for Now Don't  Change
save_best_model = SaveBestModel()
Plot_Loss=PlotLossCurves()
load_model=LoadModel()

#### Training Loop

In [112]:
start = time.time()
train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader)
print('Time Consumed', time.time()-start, "sec")

Epoch [1/50], Step [246/246], Training Loss: 0.0688
Epoch [1/50], Step [13/13], Val Loss: 0.0643

Saving best model for epoch: 1
 at C:\Users\SHAHID\shahid (ML-lab)\pro_result\1best_model.pth
Epoch [2/50], Step [246/246], Training Loss: 0.0357
Epoch [2/50], Step [13/13], Val Loss: 0.0604

Saving best model for epoch: 2
 at C:\Users\SHAHID\shahid (ML-lab)\pro_result\2best_model.pth
Epoch [3/50], Step [246/246], Training Loss: 0.0323
Epoch [3/50], Step [13/13], Val Loss: 0.0601

Saving best model for epoch: 3
 at C:\Users\SHAHID\shahid (ML-lab)\pro_result\3best_model.pth
Epoch [4/50], Step [246/246], Training Loss: 0.0295
Epoch [4/50], Step [13/13], Val Loss: 0.0602
Epoch [5/50], Step [246/246], Training Loss: 0.0271
Epoch [5/50], Step [13/13], Val Loss: 0.0586

Saving best model for epoch: 5
 at C:\Users\SHAHID\shahid (ML-lab)\pro_result\5best_model.pth
Epoch [6/50], Step [246/246], Training Loss: 0.0253
Epoch [6/50], Step [13/13], Val Loss: 0.0570

Saving best model for epoch: 6
 at C:

KeyboardInterrupt: 

#### Results

In [17]:
import torch
torch.cuda.empty_cache()

In [113]:
load_model_path=r'C:\Users\SHAHID\shahid (ML-lab)\pro_result\12best_model.pth' # Edit
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results( scaler, y_pred_scaled,test_y)

# MAPE, MAE, RMSE

New lr = 0.001
Time Consumed 0.8575103282928467 sec
Mean Absolute Error (MAE): 0.42
Median Absolute Error (MedAE): 0.36
Mean Squared Error (MSE): 0.26
Root Mean Squared Error (RMSE): 0.51
Mean Absolute Percentage Error (MAPE): 87.14 %
Median Absolute Percentage Error (MDAPE): 48.89 %


y_test_unscaled.shape=  (413, 1)
y_pred.shape=  (413, 1)


# Fine Tunning 1

In [114]:
num_epochs = 100
lr=0.0001
Batch_size = True #batch size maintained
load_model_path=r'C:\Users\SHAHID\shahid (ML-lab)\pro_result\12best_model.pth' # Edit

#### Load Model for tuning

In [115]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=32,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 0.0001


In [116]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [13/112], Step [246/246], Training Loss: 0.0244
Epoch [13/112], Step [13/13], Val Loss: 0.0529

Saving best model for epoch: 13
 at C:\Users\SHAHID\shahid (ML-lab)\pro_result\13best_model.pth


KeyboardInterrupt: 

In [64]:
load_model_path=r'C:\Users\SHAHID\shahid (ML-lab)\pro_result\11best_model.pth'
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 0.0001
Time Consumed 0.5225558280944824 sec
Mean Absolute Error (MAE): 0.37
Median Absolute Error (MedAE): 0.27
Mean Squared Error (MSE): 0.24
Root Mean Squared Error (RMSE): 0.49
Mean Absolute Percentage Error (MAPE): 59.97 %
Median Absolute Percentage Error (MDAPE): 39.58 %


y_test_unscaled.shape=  (413, 1)
y_pred.shape=  (413, 1)


# Fine Tunning 2

In [24]:
num_epochs = 20
lr = 0.00001
Batch_size=32
load_model_path=r'D:\chk6\88best_model.pth'

In [25]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [26]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [89/108], Step [2653/2653], Training Loss: 0.0001
Epoch [89/108], Step [758/758], Val Loss: 0.0001
Epoch [90/108], Step [2653/2653], Training Loss: 0.0001
Epoch [90/108], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 90
 at D:\chk6\90best_model.pth
Epoch [91/108], Step [2653/2653], Training Loss: 0.0001
Epoch [91/108], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 91
 at D:\chk6\91best_model.pth
Epoch [92/108], Step [2653/2653], Training Loss: 0.0001
Epoch [92/108], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 92
 at D:\chk6\92best_model.pth
Epoch [93/108], Step [2653/2653], Training Loss: 0.0000
Epoch [93/108], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 93
 at D:\chk6\93best_model.pth
Epoch [94/108], Step [2653/2653], Training Loss: 0.0000
Epoch [94/108], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 94
 at D:\chk6\94best_model.pth
Epoch [95/108], Step [2653/2653], Training Loss: 0.0000
Epoc

In [27]:
load_model_path=r'D:\chk6\108best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.29328155517578125 sec
Mean Absolute Error (MAE): 91.01
Median Absolute Error (MedAE): 71.54
Mean Squared Error (MSE): 14858.5
Root Mean Squared Error (RMSE): 121.9
Mean Absolute Percentage Error (MAPE): 0.62 %
Median Absolute Percentage Error (MDAPE): 0.5 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [28]:
num_epochs = 40
#lr = 0.0001
Batch_size=32
load_model_path=r'D:\chk6\108best_model.pth'

In [29]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [30]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [109/148], Step [2653/2653], Training Loss: 0.0000
Epoch [109/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 109
 at D:\chk6\109best_model.pth
Epoch [110/148], Step [2653/2653], Training Loss: 0.0000
Epoch [110/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 110
 at D:\chk6\110best_model.pth
Epoch [111/148], Step [2653/2653], Training Loss: 0.0000
Epoch [111/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 111
 at D:\chk6\111best_model.pth
Epoch [112/148], Step [2653/2653], Training Loss: 0.0000
Epoch [112/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 112
 at D:\chk6\112best_model.pth
Epoch [113/148], Step [2653/2653], Training Loss: 0.0000
Epoch [113/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 113
 at D:\chk6\113best_model.pth
Epoch [114/148], Step [2653/2653], Training Loss: 0.0000
Epoch [114/148], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 114
 at 

In [31]:
load_model_path=r'D:\chk6\148best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.7108466625213623 sec
Mean Absolute Error (MAE): 90.44
Median Absolute Error (MedAE): 70.79
Mean Squared Error (MSE): 14699.12
Root Mean Squared Error (RMSE): 121.24
Mean Absolute Percentage Error (MAPE): 0.62 %
Median Absolute Percentage Error (MDAPE): 0.49 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [32]:
num_epochs = 40
#lr = 0.00001
Batch_size=32
load_model_path=r'D:\chk6\148best_model.pth'

In [33]:
if Batch_size is True:
    model_, optimizer, start_epoch= load_model(model, model_path=load_model_path,
                               lr=lr,
                               train_X=train_X,
                               train_y=train_y,
                               validation_X=validation_X,
                               validation_y=validation_y,
                               test_X=test_X,
                               test_y=test_y)

else:
    model_, optimizer, start_epoch, train_data_loader, validation_data_loader, test_data_loader = load_model(model,model_path=load_model_path,
                                                                                             lr=lr,
                                                                                             Batch_Size=Batch_size,
                                                                                             train_X=train_X,
                                                                                             train_y=train_y,
                                                                                             validation_X=validation_X,
                                                                                             validation_y=validation_y,
                                                                                             test_X=test_X,
                                                                                             test_y=test_y)


New lr = 1e-05


In [34]:
model.load_state_dict(model_)
start = time.time()

train(start_epoch,
      num_epochs,
      best_model_path,
      fig_path,
      model,criterion,optimizer,save_best_model,Plot_Loss,
      train_data_loader,
      validation_data_loader,
     load_model_epoch=start_epoch)
print('Time Consumed', time.time()-start, "sec")

Epoch [149/188], Step [2653/2653], Training Loss: 0.0000
Epoch [149/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 149
 at D:\chk6\149best_model.pth
Epoch [150/188], Step [2653/2653], Training Loss: 0.0000
Epoch [150/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 150
 at D:\chk6\150best_model.pth
Epoch [151/188], Step [2653/2653], Training Loss: 0.0000
Epoch [151/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 151
 at D:\chk6\151best_model.pth
Epoch [152/188], Step [2653/2653], Training Loss: 0.0000
Epoch [152/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 152
 at D:\chk6\152best_model.pth
Epoch [153/188], Step [2653/2653], Training Loss: 0.0000
Epoch [153/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 153
 at D:\chk6\153best_model.pth
Epoch [154/188], Step [2653/2653], Training Loss: 0.0000
Epoch [154/188], Step [758/758], Val Loss: 0.0001

Saving best model for epoch: 154
 at 

In [35]:
load_model_path=r'D:\chk6\188best_model.pth'
test_model=TestModel()
start = time.time()
y_pred_scaled=test_model(model, test_X,load_model,load_model_path,lr)
print('Time Consumed', time.time()-start, "sec")
results(scaler, y_pred_scaled,test_y)

New lr = 1e-05
Time Consumed 0.32709288597106934 sec
Mean Absolute Error (MAE): 90.16
Median Absolute Error (MedAE): 70.6
Mean Squared Error (MSE): 14619.34
Root Mean Squared Error (RMSE): 120.91
Mean Absolute Percentage Error (MAPE): 0.61 %
Median Absolute Percentage Error (MDAPE): 0.49 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
